In [1]:
import urllib.request
import ssl
import pandas as pd
from datetime import datetime
from tkinter import *
import random
from tkinter import messagebox

## 데이터 관리 클래스 정의

In [2]:
class Finance_data:
  def __init__(self, key, values):
    self.종목 = key
    self.지수 = values["지수"]
    self.등락 = values["등락"]
    self.업다운 = values["업다운"]
    self.비율 = values["비율"]
    self.상승하락 = values["상승하락"]
    self.df = pd.DataFrame([(self.지수, self.등락, self.업다운, self.비율, self.상승하락)], columns=["지수", "등락", "업다운", "비율", "상승하락"])

  # 신규 정보 업데이트
  def update(self, 종목,values):
    tmp = pd.DataFrame([(values["지수"], values["등락"], values["업다운"], values["비율"], values["상승하락"])], columns=["지수", "등락", "업다운", "비율", "상승하락"])
    self.df = self.df.append(tmp, ignore_index=True)
    self.지수 = values["지수"]
    self.등락 = values["등락"]
    self.업다운 = values["업다운"]
    self.비율 = values["비율"]
    self.상승하락 = values["상승하락"]

  # 데이터 csv 파일로 출력
  def save_datas(self):
    
    self.df.to_csv(f"./{self.종목}_{datetime.now().strftime('%Y-%m-%d-%H-%M')}.csv")

## 크롤링 함수 정의

In [3]:
def crawling():
  context = ssl._create_unverified_context()
  #금융정보 페이지 불러오기
  # print("[금융정보 crawling]")
  URL = "https://finance.naver.com/world/"
  page = urllib.request.urlopen(URL, context=context)
  text = page.read().decode('cp949')  # 해당 페이지의 소스코드
  start = text.find("world/sise")

  # 수집정보 임시 저장 딕셔너리
  tmp_dict = dict()

  # 순차적으로 수집해서 '지수'값이 숫자가 아니면 크롤링 종료
  while True:
      span_blind = r'blind">'
      span_tag = r'<span>'
      span_end_tag=r'</span>'    
      strong_tag = r'<strong>'
      strong_end_tag = r'</strong>'
      em_tag = r'<em>'
      em_end_tag = r'</em>'
      
      #종목명
      start = text.find(span_blind,start+10)
      end =  text.find(span_end_tag,start+len(span_blind))    
      종목 = text[start+len(span_blind):end]
      #strong 지수
      start = text.find(strong_tag,end)
      end =  text.find(strong_end_tag,start)    
      지수 = text[start+len(strong_tag):end].replace(",","")
      try:
        지수 = float(지수)
      except ValueError:
        break
      
      #em 등락
      start = text.find(em_tag,end)
      end =  text.find(em_end_tag,start)    
      등락=text[start+len(em_tag):end]
      #span + -
      start = text.find(span_tag,end)
      end =  text.find(span_end_tag,start)    
      업다운=text[start+len(span_tag)*2:end]
      #span 비율
      start = end
      start = start+len(span_end_tag)
      end =  text.find(span_end_tag,start)
      비율=text[start:end]
      #blind 상승 하락
      start = text.find(span_blind,end+len(span_blind))
      end =  text.find(span_end_tag,start)    
      상승하락 = text[start+len(span_blind):end]
      tmp_dict[종목]={"지수":지수, "등락":등락, "업다운":업다운, "비율":비율, "상승하락":상승하락}
  return tmp_dict # 수집 정보 반환

## 함수 모음

In [4]:
def init_MAIN_DATAS():
    MAIN_DATAS = dict()
    crawled_data = crawling()

    for k, v in crawled_data.items():
        if k not in MAIN_DATAS.keys():
            # print(k, v)
            MAIN_DATAS[k] = Finance_data(k, v)
        else:
            # print("updated")
            MAIN_DATAS[k].update(k, v)
    return MAIN_DATAS, crawled_data
def quit():
    messagebox.showinfo("알림", "프로그램 종료을 종료합니다!")
    window.destroy()

def func_open() :
    messagebox.showinfo("메뉴선택", "열기 메뉴를 선택함")

def data_update():
    crawled_data = crawling()

    for k, v in crawled_data.items():
        if k not in MAIN_DATAS.keys():
            # print(k, v)
            MAIN_DATAS[k] = Finance_data(k, v)
        else:
            # print("updated")
            MAIN_DATAS[k].update(k, v)
    messagebox.showinfo("알림", "데이터 업데이트 완료")

def data_save():
    for k in crawled_data.keys():
        MAIN_DATAS[k].save_datas()
    messagebox.showinfo("알림", "데이터 저장 완료")

def show_detail(종목):
    data = MAIN_DATAS[종목]
    지수, 등락, 업다운, 비율, 상승하락 = data.지수, data.등락, data.업다운, data.비율, data.상승하락
    text = f"""    
        지수 : {지수}
        등락 : {등락}
        변동비율 : {업다운+비율}        
        상승하락 : {상승하락}        
        """
    messagebox.showinfo(종목,text)


# 변수 모음

In [5]:
flag_folder_path = "flags/"
flag_path = {"다우 산업":"다우.png", "니케이 225":"일본.png", "영국 FTSE 100":"영국.png", "나스닥 종합":"미국.png", "상해종합":"중국.png", "프랑스 CAC 40":"프랑스.png", "S&amp;P500":"snp.png", "항셍":"항셍.png", "독일 DAX":"독일.png"}
ZOOM_RATE =3
COLORS = ['snow', 'ghost white', 'white smoke', 'gainsboro', 'floral white', 'old lace', 'linen', 'antique white', 'papaya whip', 'blanched almond', 'bisque', 'peach puff', 'navajo white', 'lemon chiffon', 'mint cream', 'azure', 'alice blue', 'lavender', 'lavender blush', 'misty rose', 'dark slate gray', 'dim gray', 'slate gray', 'light slate gray', 'gray', 'light grey', 'midnight blue', 'navy', 'cornflower blue', 'dark slate blue', 'slate blue', 'medium slate blue', 'light slate blue', 'medium blue', 'royal blue', 'blue', 'dodger blue', 'deep sky blue', 'sky blue', 'light sky blue', 'steel blue', 'light steel blue', 'light blue', 'powder blue', 'pale turquoise', 'dark turquoise', 'medium turquoise', 'turquoise', 'cyan', 'light cyan', 'cadet blue', 'medium aquamarine', 'aquamarine', 'dark green', 'dark olive green', 'dark sea green', 'sea green', 'medium sea green', 'light sea green', 'pale green', 'spring green', 'lawn green', 'medium spring green', 'green yellow', 'lime green', 'yellow green', 'forest green', 'olive drab', 'dark khaki', 'khaki', 'pale goldenrod', 'light goldenrod yellow', 'light yellow', 'yellow', 'gold', 'light goldenrod', 'goldenrod', 'dark goldenrod', 'rosy brown', 'indian red', 'saddle brown', 'sandy brown', 'dark salmon', 'salmon', 'light salmon', 'orange', 'dark orange', 'coral', 'light coral', 'tomato', 'orange red', 'red', 'hot pink', 'deep pink', 'pink', 'light pink', 'pale violet red', 'maroon', 'medium violet red', 'violet red', 'medium orchid', 'dark orchid', 'dark violet', 'blue violet', 'purple', 'medium purple', 'thistle', 'snow2', 'snow3', 'snow4', 'seashell2', 'seashell3', 'seashell4', 'AntiqueWhite1', 'AntiqueWhite2', 'AntiqueWhite3', 'AntiqueWhite4', 'bisque2', 'bisque3', 'bisque4', 'PeachPuff2', 'PeachPuff3', 'PeachPuff4', 'NavajoWhite2', 'NavajoWhite3', 'NavajoWhite4', 'LemonChiffon2', 'LemonChiffon3', 'LemonChiffon4', 'cornsilk2', 'cornsilk3', 'cornsilk4', 'ivory2', 'ivory3', 'ivory4', 'honeydew2', 'honeydew3', 'honeydew4', 'LavenderBlush2', 'LavenderBlush3', 'LavenderBlush4', 'MistyRose2', 'MistyRose3', 'MistyRose4', 'azure2', 'azure3', 'azure4', 'SlateBlue1', 'SlateBlue2', 'SlateBlue3', 'SlateBlue4', 'RoyalBlue1', 'RoyalBlue2', 'RoyalBlue3', 'RoyalBlue4', 'blue2', 'blue4', 'DodgerBlue2', 'DodgerBlue3', 'DodgerBlue4', 'SteelBlue1', 'SteelBlue2', 'SteelBlue3', 'SteelBlue4', 'DeepSkyBlue2', 'DeepSkyBlue3', 'DeepSkyBlue4', 'SkyBlue1', 'SkyBlue2', 'SkyBlue3', 'SkyBlue4', 'LightSkyBlue1', 'LightSkyBlue2', 'LightSkyBlue3', 'LightSkyBlue4', 'SlateGray1', 'SlateGray2', 'SlateGray3', 'SlateGray4', 'LightSteelBlue1', 'LightSteelBlue2', 'LightSteelBlue3', 'LightSteelBlue4', 'LightBlue1', 'LightBlue2', 'LightBlue3', 'LightBlue4', 'LightCyan2', 'LightCyan3', 'LightCyan4', 'PaleTurquoise1', 'PaleTurquoise2', 'PaleTurquoise3', 'PaleTurquoise4', 'CadetBlue1', 'CadetBlue2', 'CadetBlue3', 'CadetBlue4', 'turquoise1', 'turquoise2', 'turquoise3', 'turquoise4', 'cyan2', 'cyan3', 'cyan4', 'DarkSlateGray1', 'DarkSlateGray2', 'DarkSlateGray3', 'DarkSlateGray4', 'aquamarine2', 'aquamarine4', 'DarkSeaGreen1', 'DarkSeaGreen2', 'DarkSeaGreen3', 'DarkSeaGreen4', 'SeaGreen1', 'SeaGreen2', 'SeaGreen3', 'PaleGreen1', 'PaleGreen2', 'PaleGreen3', 'PaleGreen4', 'SpringGreen2', 'SpringGreen3', 'SpringGreen4', 'green2', 'green3', 'green4', 'chartreuse2', 'chartreuse3', 'chartreuse4', 'OliveDrab1', 'OliveDrab2', 'OliveDrab4', 'DarkOliveGreen1', 'DarkOliveGreen2', 'DarkOliveGreen3', 'DarkOliveGreen4', 'khaki1', 'khaki2', 'khaki3', 'khaki4', 'LightGoldenrod1', 'LightGoldenrod2', 'LightGoldenrod3', 'LightGoldenrod4', 'LightYellow2', 'LightYellow3', 'LightYellow4', 'yellow2', 'yellow3', 'yellow4', 'gold2', 'gold3', 'gold4', 'goldenrod1', 'goldenrod2', 'goldenrod3', 'goldenrod4', 'DarkGoldenrod1', 'DarkGoldenrod2', 'DarkGoldenrod3', 'DarkGoldenrod4', 'RosyBrown1', 'RosyBrown2', 'RosyBrown3', 'RosyBrown4', 'IndianRed1', 'IndianRed2', 'IndianRed3', 'IndianRed4', 'sienna1', 'sienna2', 'sienna3', 'sienna4', 'burlywood1', 'burlywood2', 'burlywood3', 'burlywood4', 'wheat1', 'wheat2', 'wheat3', 'wheat4', 'tan1', 'tan2', 'tan4', 'chocolate1', 'chocolate2', 'chocolate3', 'firebrick1', 'firebrick2', 'firebrick3', 'firebrick4', 'brown1', 'brown2', 'brown3', 'brown4', 'salmon1', 'salmon2', 'salmon3', 'salmon4', 'LightSalmon2', 'LightSalmon3', 'LightSalmon4', 'orange2', 'orange3', 'orange4', 'DarkOrange1', 'DarkOrange2', 'DarkOrange3', 'DarkOrange4', 'coral1', 'coral2', 'coral3', 'coral4', 'tomato2', 'tomato3', 'tomato4', 'OrangeRed2', 'OrangeRed3', 'OrangeRed4', 'red2', 'red3', 'red4', 'DeepPink2', 'DeepPink3', 'DeepPink4', 'HotPink1', 'HotPink2', 'HotPink3', 'HotPink4', 'pink1', 'pink2', 'pink3', 'pink4', 'LightPink1', 'LightPink2', 'LightPink3', 'LightPink4', 'PaleVioletRed1', 'PaleVioletRed2', 'PaleVioletRed3', 'PaleVioletRed4', 'maroon1', 'maroon2', 'maroon3', 'maroon4', 'VioletRed1', 'VioletRed2', 'VioletRed3', 'VioletRed4', 'magenta2', 'magenta3', 'magenta4', 'orchid1', 'orchid2', 'orchid3', 'orchid4', 'plum1', 'plum2', 'plum3', 'plum4', 'MediumOrchid1', 'MediumOrchid2', 'MediumOrchid3', 'MediumOrchid4', 'DarkOrchid1', 'DarkOrchid2', 'DarkOrchid3', 'DarkOrchid4', 'purple1', 'purple2', 'purple3', 'purple4', 'MediumPurple1', 'MediumPurple2', 'MediumPurple3', 'MediumPurple4', 'thistle1', 'thistle2', 'thistle3', 'thistle4', 'gray1', 'gray2', 'gray3', 'gray4', 'gray5', 'gray6', 'gray7', 'gray8', 'gray9', 'gray10', 'gray11', 'gray12', 'gray13', 'gray14', 'gray15', 'gray16', 'gray17', 'gray18', 'gray19', 'gray20', 'gray21', 'gray22', 'gray23', 'gray24', 'gray25', 'gray26', 'gray27', 'gray28', 'gray29', 'gray30', 'gray31', 'gray32', 'gray33', 'gray34', 'gray35', 'gray36', 'gray37', 'gray38', 'gray39', 'gray40', 'gray42', 'gray43', 'gray44', 'gray45', 'gray46', 'gray47', 'gray48', 'gray49', 'gray50', 'gray51', 'gray52', 'gray53', 'gray54', 'gray55', 'gray56', 'gray57', 'gray58', 'gray59', 'gray60', 'gray61', 'gray62', 'gray63', 'gray64', 'gray65', 'gray66', 'gray67', 'gray68', 'gray69', 'gray70', 'gray71', 'gray72', 'gray73', 'gray74', 'gray75', 'gray76', 'gray77', 'gray78', 'gray79', 'gray80', 'gray81', 'gray82', 'gray83', 'gray84', 'gray85', 'gray86'] #, 'gray87', 'gray88', 'gray89', 'gray90', 'gray91', 'gray92', 'gray93', 'gray94', 'gray95', 'gray97', 'gray98', 'gray99'

In [6]:
MAIN_DATAS, crawled_data = init_MAIN_DATAS()

BUTTON_NUM=4
dataList = list(MAIN_DATAS.keys())
btnList = [""] * BUTTON_NUM
photoList=[None] * BUTTON_NUM

def imageShuffle() :
    random.shuffle(dataList)
    random.shuffle(COLORS)
    
    xPos, yPos=0, 0
    for i in range(0, BUTTON_NUM) : 
        종목 = dataList[i]
        지수 = MAIN_DATAS[종목].지수
        등락 = MAIN_DATAS[종목].등락
        업다운 = MAIN_DATAS[종목].업다운
        비율 = MAIN_DATAS[종목].비율
        상승하락 = MAIN_DATAS[종목].상승하락
        
        photoList[i] = PhotoImage(file=flag_folder_path+flag_path[종목], width=int(140*ZOOM_RATE), height=int(65*ZOOM_RATE))
        # img = img.resize((img.size[0]/2, img.size[1]/2), Image.ANTIALIAS)   
        btnList[i] = Button(window,image=photoList[i], compound='top',text=종목, bg=COLORS[i],command=lambda:show_detail(dataList[i]))
        if i % 2 == 0 :
            if i > 0 : yPos += 100*ZOOM_RATE
            xPos = 0        
        else: xPos += 140*ZOOM_RATE
        btnList[i].place(x=xPos,y=yPos, width=140*ZOOM_RATE, height=100*ZOOM_RATE)

    btn_update = Button(window, text="자료갱신", command=data_update)
    btn_update.place(x=0,y=210*ZOOM_RATE,width=70*ZOOM_RATE)

    btn_save = Button(window, text="자료저장", command=data_save)
    btn_save.place(x=70*ZOOM_RATE,y=210*ZOOM_RATE,width=70*ZOOM_RATE)

    btn = Button(window, text="뒤섞기", command=imageShuffle)
    btn.place(x=140*ZOOM_RATE,y=210*ZOOM_RATE,width=70*ZOOM_RATE)

    btn_quit = Button(window, text="종료", command=quit)
    btn_quit.place(x=210*ZOOM_RATE,y=210*ZOOM_RATE,width=70*ZOOM_RATE)
    
# 메인 코드 부분
window = Tk()
window.title("금융정보 출력 프로그램")
window.geometry(f"{int(280*ZOOM_RATE)}x{int(240*ZOOM_RATE)}")

mainMenu = Menu(window)
window.config(menu = mainMenu)
fileMenu = Menu(mainMenu)
mainMenu.add_cascade(label = "파일", menu = fileMenu)
fileMenu.add_command(label = "열기", command=func_open)
fileMenu.add_separator()
fileMenu.add_command(label = "종료", command=quit)

imageShuffle()  # 버튼이미지 표시하기

window.mainloop()

# 생성한 csv 파일 지우기

In [7]:
file_list = os.listdir()
for file_name in file_list:
    if file_name.split(".")[-1] == "csv":
        os.remove(file_name)